# Serve Placement

function to create json files for serve placement visuals

data for each serve, its outcome, placement, and serve zone data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# splits zone into two columns
def classify_zone_split(df):
    x = df['x']
    y = df['y']
    sign = x * y # if sign is pos, it's on ad side, if neg, it's deuce

    if (x < -105) or (x > 105):
        if sign > 0:
            return pd.Series(['Ad', 'Wide'])
        else:
            return pd.Series(['Deuce', 'Wide'])
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        if sign > 0:
            return pd.Series(['Ad', 'Body'])
        else:
            return pd.Series(['Deuce', 'Body'])
    elif -52.5 < x < 52.5:
        if sign > 0:
            return pd.Series(['Ad', 'T'])
        else:
            return pd.Series(['Deuce', 'T'])
    else:
        return pd.Series([np.nan, np.nan])

In [3]:
def generate_placement_jsons(df_shots, df_points):
    # only use matches with complete data
    df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])]

    # add column for winner of the point
    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', 'Detail', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

    serves = combined[(combined['Stroke'] == 'Serve') & (combined['Match Server'] == 'host')]
    serves_in = serves[serves['Result'] == 'In'].copy()

    # zone classification
    serves_in.loc[:, 'x'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in.loc[:, 'y'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
    serves_in[['side', 'serveInPlacement']] = serves_in.apply(classify_zone_split, axis=1)

    # modify coordinates based on the y-value
    serves_in['x'] = np.where(serves_in['y'] < 0, -serves_in['x'], serves_in['x'])
    serves_in['y'] = np.where(serves_in['y'] < 0, -serves_in['y'], serves_in['y'])

    # add serve outcome
    serves_in['serveOutcome'] = serves_in['Point Winner'].apply(lambda x: 'Won' if x == 'host' else 'Lost')
    serves_in['serveOutcome'] = np.where(serves_in['Detail'] == 'Ace', 'Ace', serves_in['serveOutcome'])

    # rename some columns to match json
    serves_in = serves_in.rename(columns={'Point': 'pointNumber', 'Player': 'serverName'})

    # only get required columns
    placement = serves_in[['pointNumber', 'serverName', 'x', 'y', 'side', 'serveInPlacement', 'serveOutcome']]
    placement.to_json('serve_place.json', orient='records') # save json



    # LABELS JSON

    # group by side and serveInPlacement, and calculate count and serves won
    distribution = serves_in.groupby(['side', 'serveInPlacement']).agg(
        count=('pointNumber', 'size'),
        serves_won=('Point Winner', lambda x: (x == 'host').sum())
    ).reset_index()

    # calculate the win percentage (proportion)
    distribution['proportion'] = distribution['serves_won'] / distribution['count']

    # find the min and max proportions
    min_proportion = distribution['proportion'].min()
    max_proportion = distribution['proportion'].max()

    # create labels df and determine if each value is max, min, or neither
    labels = distribution.copy()
    labels['proportion_label'] = (labels['proportion'] * 100).round(1).astype(str) + "%"
    labels['count_label'] = labels['count']

    labels['x'] = np.where(
        (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'Wide'), 131.25,
        np.where(
            (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'Body'), 78.75,
            np.where(
                (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'T'), 26.25,
                np.where(
                    (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'T'), -26.25,
                    np.where(
                        (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'Body'), -78.75,
                        np.where(
                            (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'Wide'), -131.25,
                            np.nan
                        )
                    )
                )
            )
        )
    )

    # determine max/min status
    labels['max_min'] = np.where(
        labels['proportion'] == max_proportion, "max",
        np.where(labels['proportion'] == min_proportion, "min", "no")
    )

    # save labels to json
    labels.to_json('serve_place_labels.json', orient='records')

    return

In [4]:
player = "Rudy Quan"
path = f"../../data/mens/{player}/combined.xlsx"
df_shots = pd.read_excel(path, sheet_name="Shots")
df_points = pd.read_excel(path, sheet_name="Points")
generate_placement_jsons(df_shots, df_points)